### 1. Bibliotecas

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from IPython import display

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.tree import export_graphviz

from pandas.plotting import scatter_matrix

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### 2. Import do Dataset

In [4]:
col_names = ['Class', 'Specimen_Number', 'Eccentricity',
             'Aspect_Radio', 'Elongation', 'Solidity',
             'Stochastic_Convexity', 'Isoperimetric_Factor', 'Maximal_Identation_Depth',
             'Lobedness', 'Average_Intensity', 'Average_Contrast', 'Smoothness',
             'Third_Moment', 'Uniformity', 'Entropy']

#df = pd.read_csv('dados/leaf.csv', header=None, names=col_names)
df = pd.read_csv('dados/leaf.csv', header=None)

In [30]:
df.head(100)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,1,0.72694,1.4742,0.32396,0.98535,1.00000,0.83592,0.004657,0.003947,0.047790,0.127950,0.016108,0.005232,0.000275,1.17560
1,1,2,0.74173,1.5257,0.36116,0.98152,0.99825,0.79867,0.005242,0.005002,0.024160,0.090476,0.008119,0.002708,0.000075,0.69659
2,1,3,0.76722,1.5725,0.38998,0.97755,1.00000,0.80812,0.007457,0.010121,0.011897,0.057445,0.003289,0.000921,0.000038,0.44348
3,1,4,0.73797,1.4597,0.35376,0.97566,1.00000,0.81697,0.006877,0.008607,0.015950,0.065491,0.004271,0.001154,0.000066,0.58785
4,1,5,0.82301,1.7707,0.44462,0.97698,1.00000,0.75493,0.007428,0.010042,0.007938,0.045339,0.002051,0.000560,0.000024,0.34214
5,1,6,0.72997,1.4892,0.34284,0.98755,1.00000,0.84482,0.004945,0.004451,0.010487,0.058528,0.003414,0.001125,0.000025,0.34068
6,1,7,0.82063,1.7529,0.44458,0.97964,0.99649,0.76770,0.005928,0.006395,0.018375,0.080587,0.006452,0.002271,0.000041,0.53904
7,1,8,0.77982,1.6215,0.39222,0.98512,0.99825,0.80816,0.005099,0.004731,0.024875,0.089686,0.007979,0.002466,0.000147,0.66975
8,1,9,0.83089,1.8199,0.45693,0.98240,1.00000,0.77106,0.006005,0.006564,0.007245,0.040616,0.001647,0.000388,0.000033,0.33696
9,1,10,0.90631,2.3906,0.58336,0.97683,0.99825,0.66419,0.008402,0.012848,0.007010,0.042347,0.001790,0.000459,0.000028,0.28082


In [5]:
X = df.iloc[:, 2:] # 14 feature columns
y = df.iloc[:, 0]  # 1 class column

## 3. Visualização dos Dados

## 4. Treinamento do Modelo

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rfc = RandomForestClassifier(random_state=42)

# Dictionary with parameters names (string) as keys and lists of parameter settings to try as values,
# or a list of such dictionaries, in which case the grids spanned by each dictionary in the list are explored.
# This enables searching over any sequence of parameter settings.

max_depth = [5, 10, 15, 18, 20]
max_depth.append(None)

param_grid = {
    # The number of trees in the forest
    'n_estimators': [25, 50, 100, 150, 200, 250],
    
    # The function to measure the quality of a split
    'criterion' :['gini', 'entropy'],
    
    # The maximum depth of the tree
    'max_depth' : max_depth,
    
    # The number of features to consider when looking for the best split
    'max_features': ['auto', 'sqrt', 'log2']
}

scoring = ['accuracy', 'f1_micro', 'f1_macro']

CV_rfc = GridSearchCV(estimator=rfc,
                      scoring=scoring,
                      n_jobs=-1, # Number of jobs to run in parallel (-1 == All)
                      param_grid=param_grid,
                      verbose=True,
                      refit='accuracy',
                      cv=10, # 10-Fold Cross-Validation
                      return_train_score=False)
CV_rfc.fit(X_train, y_train)

df2 = pd.DataFrame(CV_rfc.cv_results_)
df2.to_csv('resultado.csv', sep=';')

Fitting 10 folds for each of 216 candidates, totalling 2160 fits


c:\users\mathe\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:   52.4s
[Parallel(n_jobs=-1)]: Done 2160 out of 2160 | elapsed:  1.2min finished
c:\users\mathe\miniconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will b

In [22]:
# Get the best parameters
print ('Best parameters:', CV_rfc.best_params_)
print ('Best Score:', CV_rfc.best_score_)

Best parameters: {'criterion': 'gini', 'max_depth': 18, 'max_features': 'auto', 'n_estimators': 200}
Best Score: 0.7904411764705882


In [23]:
rfc1=RandomForestClassifier(random_state=42,
                            max_features='auto',
                            n_estimators= 200,
                            max_depth=18,
                            criterion='gini')

In [24]:
rfc1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=18, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [25]:
y_pred=rfc1.predict(X_test)

In [26]:
print("Accuracy for Random Forest on Test data: ", accuracy_score(y_test,pred))

Accuracy for Random Forest on Test data:  0.7794117647058824


In [27]:
# Pq não aparecem as 40 classes?
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           1     0.5000    0.6667    0.5714         3
           2     1.0000    1.0000    1.0000         1
           3     0.0000    0.0000    0.0000         1
           4     0.0000    0.0000    0.0000         2
           5     1.0000    1.0000    1.0000         3
           6     1.0000    1.0000    1.0000         2
           7     0.5000    1.0000    0.6667         1
           8     1.0000    1.0000    1.0000         5
           9     0.6667    0.6667    0.6667         3
          10     0.5000    1.0000    0.6667         1
          11     1.0000    1.0000    1.0000         7
          12     1.0000    0.5000    0.6667         2
          13     0.3333    0.5000    0.4000         2
          14     1.0000    0.5000    0.6667         2
          15     1.0000    1.0000    1.0000         1
          22     0.8000    0.8000    0.8000         5
          24     1.0000    1.0000    1.0000         2
          25     1.0000    

c:\users\mathe\miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\mathe\miniconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## 5. Visualização dos Resultados

In [28]:
# Plot do trab. anterior não funciona bem com 40 classes
cm = confusion_matrix(y_test,pred)
classes = ['Quercus_suber','Salix_atrocinera','Populus_nigra','Alnus_sp.','Quercus_robur',
          'Crataegus_monogyna','Ilex_aquifolium','Nerium_oleander','Betula_pubescens', 'Tilia tomentosa',
          'Acer_palmatum', 'Celtis_sp.', 'Corylus_avellana','Castanea_sativa','Populus_alba',
          'Acer_negundo','Taxus_bacatta','Papaver_sp.','Polypolium_vulgare','Pinus_sp.',
          'Fraxinus_sp.','Primula_vulgaris','Erodium_sp.','Bougainvillea_sp.','Arisarum_vulgare',
          'Euonymus_japonicus','Ilex_perado_ssp._azorica','Magnolia_soulangeana','Buxus_sempervirens','Urtica_dioica',
          'Podocarpus_sp.','Acca_sellowiana','Hydrangea_sp.','Pseudosasa_japonica','Magnolia_grandiora',
           'Geranium_sp.','Aesculus_californica','Chelidonium_majus',
           'Schinus_terebinthifolius','Fragaria_vesca']
print (cm)

[[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 

In [21]:
str_tree = export_graphviz(rfc1.estimators_[0], 
   out_file='tree.dot', 
   feature_names=X_train.columns, # column names
   class_names=True)

os.system('dot -Tpng tree.dot -o tree.png')
# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

<class 'str'>
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36


FileNotFoundError: [Errno 2] No such file or directory: 'tree.png'

---